In [4]:
import pandas as pd

In [5]:
# !pip install lyricsgenius
from lyricsgenius import Genius
import pandas as pd
import numpy as np

genius = Genius(
    "7Ise90s_tMVx4jQroYGWIOaskQOTBFiCh3G9J8CtgLKBK65cXk3HNFEVMYNMoGBd")


In [7]:
charts = pd.read_csv('datasets/charts.csv')
songs_and_artist = charts[['song', 'weeks-on-board', 'artist']].groupby(
    ['song', 'artist']).max().sort_values(by='weeks-on-board', ascending=False).head(50)
songs_and_artist.reset_index(level=['song', 'artist'], inplace=True)
songs_and_artist.head()

,song,artist,weeks-on-board
0,Radioactive,Imagine Dragons,87
1,Blinding Lights,The Weeknd,83
2,Sail,AWOLNATION,79
3,I'm Yours,Jason Mraz,76
4,How Do I Live,LeAnn Rimes,69


In [29]:
import requests
URL = 'https://api.musixmatch.com/ws/1.1/track.search?'
api_key = 'c446c53140147b0285b00171e61b09ca'


def search(song, artist):
    params = {
        'q': (song + ' ' + artist).replace('+', ' '),
        'apikey': api_key
    }
    response = requests.get(URL, params=params)
    print(response.url)
    return response.text


In [30]:
songs_and_artist['Query'] = songs_and_artist.apply(lambda x: search(x.song, x.artist),axis=1)

https://api.musixmatch.com/ws/1.1/track.search?q=Radioactive+Imagine+Dragons&apikey=c446c53140147b0285b00171e61b09ca
https://api.musixmatch.com/ws/1.1/track.search?q=Blinding+Lights+The+Weeknd&apikey=c446c53140147b0285b00171e61b09ca
https://api.musixmatch.com/ws/1.1/track.search?q=Sail+AWOLNATION&apikey=c446c53140147b0285b00171e61b09ca
https://api.musixmatch.com/ws/1.1/track.search?q=I%27m+Yours+Jason+Mraz&apikey=c446c53140147b0285b00171e61b09ca
https://api.musixmatch.com/ws/1.1/track.search?q=How+Do+I+Live+LeAnn+Rimes&apikey=c446c53140147b0285b00171e61b09ca
https://api.musixmatch.com/ws/1.1/track.search?q=Party+Rock+Anthem+LMFAO+Featuring+Lauren+Bennett+%26+GoonRock&apikey=c446c53140147b0285b00171e61b09ca
https://api.musixmatch.com/ws/1.1/track.search?q=Counting+Stars+OneRepublic&apikey=c446c53140147b0285b00171e61b09ca
https://api.musixmatch.com/ws/1.1/track.search?q=Foolish+Games%2FYou+Were+Meant+For+Me+Jewel&apikey=c446c53140147b0285b00171e61b09ca
https://api.musixmatch.com/ws/1.1/t

In [64]:
import json

result = json.loads(songs_and_artist.Query[2])

id, genre = list(), list()
for track in result['message']['body']['track_list']:
    try:
        print(track['track']['commontrack_id'])
        print(track['track']['primary_genres']['music_genre_list'][0]['music_genre']['music_genre_name'])
        id.append(track['track']['commontrack_id'])
        genre.append(track['track']['primary_genres']
                    ['music_genre_list'][0]['music_genre']['music_genre_name'])
    except:
        genre.append(None)
        id.append(None)

print(id, genre)


37894156
Heavy Metal
26046854
Rock
26050299
Pop
33428680
Pop
55054558
58037525
48496023
19097137
Pop
33652690
Karaoke
[37894156, 26046854, 26050299, 33428680, None, None, None, 19097137, 33652690] ['Heavy Metal', 'Rock', 'Pop', 'Pop', None, None, None, 'Pop', 'Karaoke']


In [63]:
songs_and_artist.Query[2]

'{"message":{"header":{"status_code":200,"execute_time":0.27349019050598,"available":116},"body":{"track_list":[{"track":{"track_id":69823139,"track_name":"Sail (Awolnation Cover)","track_name_translation_list":[],"track_rating":1,"commontrack_id":37894156,"instrumental":0,"explicit":0,"has_lyrics":0,"has_subtitles":0,"has_richsync":0,"num_favourite":0,"album_id":19206246,"album_name":"The Nightmare Within","artist_id":27386715,"artist_name":"Phoenix Anstieg","track_share_url":"https:\\/\\/www.musixmatch.com\\/lyrics\\/Phoenix-Anstieg\\/Sail-Awolnation-Cover?utm_source=application&utm_campaign=api&utm_medium=","track_edit_url":"https:\\/\\/www.musixmatch.com\\/lyrics\\/Phoenix-Anstieg\\/Sail-Awolnation-Cover\\/edit?utm_source=application&utm_campaign=api&utm_medium=","restricted":0,"updated_time":"2014-03-13T17:20:04Z","primary_genres":{"music_genre_list":[{"music_genre":{"music_genre_id":1153,"music_genre_parent_id":21,"music_genre_name":"Heavy Metal","music_genre_name_extended":"Rock

In [105]:
songs_and_artist.describe()

,weeks-on-board
count,50.000000
mean,59.640000
std,7.782135
min,53.000000
25%,54.000000
50%,57.000000
75%,61.750000
max,87.000000


In [106]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize  
seed = 42

In [107]:
# drop songs of which lyrics can't be found  
songs_and_artist_1 = songs_and_artist[songs_and_artist['lyrics'] != 'empty'] 
songs_and_artist_1

,song,artist,weeks-on-board,lyrics
1,Blinding Lights,The Weeknd,83,[Intro]\nYeah\n\n[Verse 1]\nI've been tryna ca...
2,Sail,AWOLNATION,79,[Intro]\nSail!\n\n[Verse 1]\nThis is how I sho...
3,I'm Yours,Jason Mraz,76,[Verse 1]\nWell you done done me in; you bet I...
4,How Do I Live,LeAnn Rimes,69,[Verse 1]\nHow do I get through one night with...
5,Party Rock Anthem,LMFAO Featuring Lauren Bennett & GoonRock,68,[Intro: Redfoo]\nParty Rock\nYeah\nWoo!\nLet's...
6,Counting Stars,OneRepublic,68,"[Intro]\nLately, I've been, I've been losing s..."
7,Foolish Games/You Were Meant For Me,Jewel,65,Part III\n\nLIGHTS flicked on and house-doors ...
8,Rolling In The Deep,Adele,65,[Verse 1]\nThere's a fire starting in my heart...
9,Before He Cheats,Carrie Underwood,64,"[Verse 1]\nRight now, he's probably slow danci..."
10,Ho Hey,The Lumineers,62,[Intro]\nHo!\nHey!\nHo!\nHey!\nHo!\n\n[Verse 1...


In [124]:

# Define X and y
X = songs_and_artist_1['lyrics'].to_list()
y = songs_and_artist_1['weeks-on-board'].to_list()


# split train and test set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=seed)


# Initialize pipeline
pipe = Pipeline([('count_vec', CountVectorizer(stop_words="english", analyzer='word',
                                               ngram_range=(1, 1), max_df=1.0,
                                               min_df=1, max_features=None)),
                 ('svm', LinearSVC())])


pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)


In [123]:
from sklearn.metrics import plot_confusion_matrix , classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          53       0.00      0.00      0.00       1.0
          54       0.00      0.00      0.00       2.0
          55       0.00      0.00      0.00       0.0
          56       0.00      0.00      0.00       2.0
          57       0.00      0.00      0.00       2.0
          58       0.00      0.00      0.00       1.0
          59       0.00      0.00      0.00       0.0
          61       0.00      0.00      0.00       1.0
          62       0.00      0.00      0.00       0.0
          68       0.00      0.00      0.00       1.0
          69       0.00      0.00      0.00       0.0
          83       0.00      0.00      0.00       0.0

    accuracy                           0.00      10.0
   macro avg       0.00      0.00      0.00      10.0
weighted avg       0.00      0.00      0.00      10.0



/Users/yc/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yc/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yc/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yc/opt/anaconda3/lib/python3.

In [129]:
len(y_test)

10